In [1]:
import sys
from time import sleep
import urllib.request as ur

from rich import print

sys.path.append('..')
from base import BaseCralwer
from utils import parsingFile, getLogger, page_scroll, tm2int
from urls import TMON_LOGIN_URL, TMON_SEARCH_URL
from selenium.common.exceptions import NoSuchElementException

In [2]:
class Tmon(BaseCralwer):
    def __init__(self): # init, login, delete alert
        super().__init__()
        self.log.info("TMON Observer Starting...")
        self.products = []

    def _login(self):
        key = parsingFile('../config/login')['tmon']
        driver = self.get_driver()
        driver.get(TMON_LOGIN_URL)
        driver.find_element_by_id('userid').send_keys(key['id'])
        driver.find_element_by_id('pwd').send_kkjeys(key['pw'])
        driver.find_element_by_link_text('로그인').click()
        driver.get('http://search.tmon.co.kr') 
        # 오늘 하루 보지 않기, 1번 클릭하면 모든 팝업 다 지워짐
        driver.find_element_by_class_name("expires").click()

    def search_products(self, product_keyword, low, high):
        query = TMON_SEARCH_URL.format(product_keyword)
        scrolls = 0
        driver = self.get_driver()
        driver.get(query)
        # 내림차순
        driver.find_element_by_xpath( \
            '//*[@id="search_app"]/div[2]/div[3]/div[2]/dl/dd[4]/a' \
        ).click()
        sleep(1)
        low, high = sorted([low, high])
        
        while True:
            links = driver.find_elements_by_css_selector('ul > div > div > li > a')
            prices = driver.find_elements_by_css_selector('span.price > span.sale > i')
            
            # 상품 자체가 부족할 경우
            if scrolls > 199999:
                break
            
            if (tm2int(prices[-1].text)) > low:
                scrolls += 10000
                page_scroll(driver, scrolls)
                sleep(0.5)
                continue
                
            for link, price in zip(links, prices):
                price = tm2int(price.text)
                # high, low 값이 같을 때 check
                if price > high:
                    continue
                if price < low: # 마지막 if
                    break
                self.products.append(link.get_attribute('href'))
            break
    
    def check_tmon_product(self, link):
        pass
    
    
    def check_products(self):
        self.baskets = [link for link in self.products \
                        if self.check_tmon_product(link)]
    
    def check_urls(self):
        pass
    
    def _check_price(self, low, high):
        driver = self.get_driver()
        links = driver.find_elements_by_css_selector('ul > div > div > li > a')
        prices = driver.find_elements_by_css_selector('span.price > span.sale > i')
        self.basket = []
        for link_driver, price_driver in zip(links, prices):
            link = link_driver.get_attribute('href') 
            price = int(price_driver.text.replace(',',''))
            if price < low or price > high:
                continue
            self.basket.append((link, price))

    def _check_product(self, low, high):
        driver = self.get_driver()
        for link, price in self.basket:
            print(link)
            driver.get(link)
            driver.find_element_by_css_selector('#_optionScroll > div > \
                    div > div > div > button').click()
            contents = driver.find_elements_by_css_selector('#_optionScroll > \
                    div > div > div > div > ul > li')
            user_product = []
            for content in contents:
                if content.text == '':
                    break
                # price wrong..
                price = int(price_match.match(content.text).group(1).replace(',',''))
                is_valid = not content.text.endswith('매진')
                if is_valid and (low <= price <= high):
                    user_product.append(content.text)     
            print('이 link 에서 구매 가능한 물품')
            print('\n'.join(user_product))
                    
    def _get_price(self): # price가 있다는 건 품절이 아니라는 것을 의미
        driver = self.get_driver()
        driver.get("http://www.tmon.co.kr/deal/2471909782?keyword=%EB%8B%8C%ED%85%90%EB%8F%84+%EC%8A%A4%EC%9C%84%EC%B9%98&is_ad=Y&partner_srl=943378")
        options = driver.find_elements_by_class_name("dep-sel")
        options_len = len(options) // 2
        options[0].click()  # first option open
        idx = 0
        link_product = []
        while(idx < options_len):
            for options in driver.find_elements_by_css_selector( \
                    '#_optionScroll > div > div > div > div.dep-sel.dep{}.open > ul > li'.format(idx)):
                if options.get_attribute('class') == 'soldout':
                    continue
                else:
                    link_product.append(options.text)
                    idx += 1
                    break
            idx += 1
        print(link_product)
        
    def run(self):
        self._login()
        products = parsingFile('../config/product.cfg')
        print(products)
        for prod in products.values():
            self._search(prod['keyword'])
            print('search complete')
            self._check_price(prod['low_price'], prod['high_price'])
            print('check_price complete')
            self._check_product(prod['low_price'], prod['high_price'])
            print('check_product complete')


In [9]:
a = Tmon()

In [20]:
products= parsingFile('../config/product')
print(products)

In [5]:
a.products

[]

In [6]:
a._login()

In [7]:
a.search_product(products['product1']['keyword'], products['product1']['low_price'],
                products['product1']['high_price'])

In [10]:
len(a.driver.find_elements_by_class_name("item"))

72

In [14]:
a.driver.find_elements_by_class_name("item")[-4].text

'스위치 액세서리 번들닌텐도 스위치 용 Orzly Geek Pack : 케이\n균일가\n254,400원'

In [19]:
len(a.products), a.products[-1]

(14,
 'http://www.tmon.co.kr/deal/2598947130?keyword=%EB%8B%8C%ED%85%90%EB%8F%84+%EC%8A%A4%EC%9C%84%EC%B9%98&tl_area=SALDEAL&tl_ord=64&searchClick=DL%7CND%7COMM&thr=ts')

In [16]:
prices = a.driver.find_elements_by_css_selector('span.price > span.sale > i')

In [27]:
prices[-1].text

'252,500'

# Tmon Check

In [6]:
driver = Tmon()

In [7]:
driver._login()

In [18]:
driver.driver.get(TMON_SEARCH_URL.format('닌텐도 스위치'))

In [22]:
driver.search_products(products['product1']['keyword'], products['product1']['low_price'],
                products['product1']['high_price'])

In [36]:
driver.driver.get(driver.products[6])

## Options Solution

In [26]:
# case 1: Option 존재 길이 1
driver.driver.get(driver.products[0])

In [32]:
# case 2: Option 존재 하나 선택되어짐
driver.driver.get(driver.products[1])

## case 2

In [33]:
driver.driver.find_elements_by_class_name("dep-sel")

[]

In [41]:
driver.driver.find_element_by_css_selector("ul.prod > li > span.tit").text

'닌텐도 스위치 해외버전 라이트 자시안 자마젠타 에디션[편의점 픽업 불가] 998개 남음'

In [44]:
driver.driver.find_element_by_css_selector("div.price_area > div > div > span > strong").text

'359,000'

In [30]:
# case 3: Option 존재 길이 2
driver.driver.get(driver.products[2])

In [45]:
# case 3: Option 존재 길이 3
driver.driver.get(driver.products[9])

In [48]:
# test case:
driver.driver.get('http://www.tmon.co.kr/deal/3754589658?keyword=%EB%AA%A8%EC%9D%8C&tl_area=SALDEAL&tl_ord=6&searchClick=DL%7CND%7CBM&thr=ts')

3

In [29]:
a = driver_selector(options_css_selector.format(2))[options_select_idx[2]]

In [30]:
a.is_enabled()

True

In [49]:
driver_selector = driver.driver.find_elements_by_css_selector
options_css_selector = '#_optionScroll > div > div > div > div.dep-sel.dep{} > ul > li'
options_maxlen = [len(driver.driver.find_elements_by_css_selector(options_css_selector.format(0)))//2]
options_len = len(driver.driver.find_elements_by_class_name("dep-sel")) // 2
options_select_idx = [0 for _ in range(options_len)]
options = driver.driver.find_elements_by_class_name("dep-sel")[:options_len]
# print(options_maxlen)
# print(options_select_idx)
options[0].click()

products = []
prices = []
options_click_idx = 0

while True:
    # product complete
    if options_click_idx == options_len:
#         print('section 2')
        product_price, product_title = None, None
        while not (product_price and product_title):
            try:
                product_price = driver.driver.find_element_by_css_selector('div.price_area > div > div > span > strong')
                product_title = driver.driver.find_element_by_xpath('//*[@id="_optionScroll"]/div/ul[1]/li/span[1]')
            except NoSuchElementException:
#                 print('except here')
                options[options_click_idx-1].click()
                sleep(0.2)
                driver_selector(options_css_selector.format( \
                     options_click_idx-1))[options_select_idx[options_click_idx-1]-1].click()
                
        print(product_title.text)
        products.append(product_title.text)
        prices.append(product_price.text)
        options_click_idx -= 1
        driver.driver.find_element_by_class_name("del").click()
        options[options_click_idx].click()
        continue
    
    # option maxlen
    if options_select_idx[options_click_idx] == 0 and options_click_idx != 0:
#         print('section4')
        options_maxlen.append(len(driver_selector(options_css_selector.format(options_click_idx)))//2)
        
    # option arrived maxlen
#     print(options_select_idx, options_maxlen)
    if options_select_idx[options_click_idx] == options_maxlen[options_click_idx]:
#         print('section 3')
        options_select_idx[options_click_idx] = 0
        options_click_idx -= 1
        options[options_click_idx].click()
        options_maxlen.pop()
        if options_maxlen:
            continue
        break
            
#     print('section 5')    
#     print(options_click_idx, options_select_idx[options_click_idx])
    # option:n click
    n_option = None
    while not n_option:
        try:
            n_option = driver_selector(options_css_selector.format(options_click_idx))[options_select_idx[options_click_idx]]
        except NoSuchElementException:
            pass
        
    # option:n plus
    options_select_idx[options_click_idx] += 1
    
    if n_option.get_attribute('class') == 'soldout':    
        continue
        
    n_option.click()
    
    # option:n to n + 1
    options_click_idx += 1
    
print(prices)
print(products)

ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [50]:
prices

['12,900',
 '12,900',
 '12,900',
 '12,900',
 '12,900',
 '12,900',
 '5,900',
 '5,900',
 '5,900',
 '5,900',
 '5,900',
 '5,900',
 '5,900',
 '5,900',
 '5,900',
 '5,900',
 '5,900',
 '12,900',
 '12,900',
 '12,900',
 '12,900',
 '12,900',
 '12,900',
 '12,900',
 '5,900',
 '5,900',
 '5,900',
 '5,900',
 '5,900',
 '5,900',
 '5,900',
 '5,900',
 '5,900',
 '12,900',
 '12,900',
 '12,900',
 '12,900',
 '5,900',
 '5,900',
 '5,900',
 '12,900',
 '12,900',
 '12,900']